In [38]:
# Import libraries
import random
import numpy as np
import matplotlib.pyplot as plt

import sklearn
import sklearn.model_selection
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier

import tensorflow as tf 
import keras
import keras.regularizers
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import BatchNormalization, GaussianNoise, GaussianDropout, LeakyReLU
from keras.wrappers.scikit_learn import KerasClassifier

## Loading Dataset

In [2]:
# Load final validation data set
FV_data = np.loadtxt('../data/test_data.txt',delimiter=' ',skiprows=1)
# Load input training data set
train_data = np.loadtxt('../data/training_data.txt',delimiter=' ',skiprows=1)

# Split y_train and x_train from training set
X_Tall = train_data[:,1:]
y_Tall = train_data[:,0]

In [73]:
kf = StratifiedKFold(n_splits=5)
inds = list(kf.split(X_Tall, y_Tall))

train_idx, val_idx = inds[4]

X_train, y_train = X_Tall[train_idx], y_Tall[train_idx] 
X_val, y_val = X_Tall[val_idx], y_Tall[val_idx]

In [74]:
# Function to generate DNN of given depth and width
def create_model(layers,Pdrop):
    model = Sequential()
    model.add(Dense(layers[0],input_shape=(1000,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.3))
    for i in layers[1:]:
        model.add(Dense(i))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.3))
        model.add(Dropout(Pdrop))
    
    # predicting probabilities of each of the 2 classes
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    model.compile(loss='binary_crossentropy',optimizer='RMSprop', metrics=['accuracy'])
    return model

neural = KerasClassifier(build_fn=create_model, layers=[500,250,125,75,25], Pdrop=0.4)

In [75]:
neural.fit(X_train, y_train, batch_size=2**8, epochs=6, validation_data=(X_val, y_val))

Train on 16001 samples, validate on 3999 samples
Epoch 1/6
16001/16001 [==============================] - 4s 269us/step - loss: 0.5028 - acc: 0.7619 - val_loss: 0.3809 - val_acc: 0.8452
Epoch 2/6
16001/16001 [==============================] - 2s 116us/step - loss: 0.3514 - acc: 0.8507 - val_loss: 0.4069 - val_acc: 0.8520
Epoch 3/6
16001/16001 [==============================] - 2s 119us/step - loss: 0.3157 - acc: 0.8693 - val_loss: 0.4102 - val_acc: 0.8512
Epoch 4/6
16001/16001 [==============================] - 2s 119us/step - loss: 0.2897 - acc: 0.8812 - val_loss: 0.4344 - val_acc: 0.8470
Epoch 5/6
16001/16001 [==============================] - 2s 118us/step - loss: 0.2631 - acc: 0.8908 - val_loss: 0.4488 - val_acc: 0.8520
Epoch 6/6
16001/16001 [==============================] - 2s 119us/step - loss: 0.2387 - acc: 0.9034 - val_loss: 0.4300 - val_acc: 0.8482


In [76]:
svm = make_pipeline(StandardScaler(), SVC(C=4.0, gamma=0.0002))
svm.fit(X_train, y_train)
print(svm.score(X_train, y_train))
print(svm.score(X_val, y_val))

0.9120054996562715
0.8549637409352339


In [77]:
ada = make_pipeline(StandardScaler(),
                     AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), 
                                        n_estimators=500,
                                        learning_rate=1))

ada.fit(X_train, y_train)
print(ada.score(X_train, y_train))
print(ada.score(X_val, y_val))

0.8669458158865071
0.8402100525131283


In [78]:
pred_nn = neural.predict(X_val)
pred_svm = svm.predict(X_val)
pred_ada = ada.predict(X_val)

In [79]:
pred_comb = np.zeros(len(y_val))
for i in range(len(y_val)):
    votes = [pred_nn[i][0], pred_svm[i], pred_ada[i]]
    ones= votes.count(1)
    if ones >= 2:
        pred_comb[i] = 1
    else:
        pred_comb[i] = 0
        
accuracy_score(pred_comb, y_val)

0.8534633658414603

In [ ]:
# Val for first  fold: 0.8557860534866284
# Val for second fold: 0.8512871782054486
# Val for third  fold: 0.843
# Val for fourth fold: 0.8472118029507377
# Val for fifth  fold: 0.8534633658414603
# Average: 0.850